In [3]:
!git clone https://github.com/eBay/RANSynCoders.git

Cloning into 'RANSynCoders'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 105 (delta 19), reused 9 (delta 8), pack-reused 70
Receiving objects: 100% (105/105), 55.76 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [111]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input, Embedding, multiply, BatchNormalization
from keras.models import Model, Sequential
from keras.layers import Reshape, Dense, Dropout, Flatten
from keras.layers import LeakyReLU
from keras.layers import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix


In [112]:
data = '/content/RANSynCoders/data'
train = pd.read_csv(os.path.join(data,"train.csv"))
x_test = pd.read_csv(os.path.join(data,'test.csv'))
y_test = pd.read_csv(os.path.join(data,'test_label.csv'))

In [113]:
x_test

,timestamp_(min),feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24
0,132480.0,0.775374,0.909185,0.606704,0.660626,0.449968,0.426717,0.471591,0.434668,0.479511,...,0.400617,0.480444,0.588670,0.404036,0.638957,0.020236,0.042705,0.173375,0.008715,0.105991
1,132481.0,0.775423,0.909142,0.607796,0.660655,0.487813,0.442545,0.505682,0.448640,0.499451,...,0.396902,0.500213,0.587041,0.421605,0.637865,0.018550,0.042705,0.191826,0.010893,0.110599
2,132482.0,0.775458,0.909004,0.607988,0.660623,0.469532,0.433682,0.477273,0.450906,0.487223,...,0.389270,0.487982,0.582404,0.407393,0.636091,0.018550,0.039146,0.188361,0.010893,0.115207
3,132483.0,0.775513,0.909200,0.607218,0.660640,0.459910,0.435581,0.482955,0.454683,0.490431,...,0.403780,0.491253,0.584660,0.407141,0.638138,0.020236,0.042705,0.186975,0.013072,0.110599
4,132484.0,0.775587,0.909318,0.606654,0.660643,0.462155,0.436214,0.477273,0.449396,0.494671,...,0.403636,0.495947,0.581527,0.402761,0.632678,0.021922,0.042705,0.156325,0.008715,0.110599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87836,220316.0,0.864548,0.660041,0.590923,0.413322,0.364657,0.380184,0.494318,0.450529,0.382934,...,0.394973,0.386574,0.755608,0.355207,0.835244,0.000000,0.028470,0.164872,0.021786,0.069124
87837,220317.0,0.864483,0.536428,0.590742,0.413331,0.363695,0.378601,0.500000,0.451662,0.377564,...,0.386467,0.381169,0.562101,0.349141,0.490718,0.000000,0.010676,0.157812,0.015251,0.069124
87838,220318.0,0.864644,0.636236,0.590451,0.413619,0.348942,0.377018,0.488636,0.438822,0.373536,...,0.413731,0.377329,0.640556,0.342261,0.700519,0.000000,0.028470,0.163298,0.015251,0.129032
87839,220319.0,0.864553,0.643499,0.590222,0.413689,0.355997,0.379550,0.522727,0.448640,0.374963,...,0.416956,0.378751,0.759744,0.343052,0.828556,0.000000,0.032028,0.150160,0.017429,0.059908


In [114]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132481 entries, 0 to 132480
Data columns (total 26 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   timestamp_(min)  132481 non-null  float64
 1   feature_0        132481 non-null  float64
 2   feature_1        132481 non-null  float64
 3   feature_2        132481 non-null  float64
 4   feature_3        132481 non-null  float64
 5   feature_4        132279 non-null  float64
 6   feature_5        132232 non-null  float64
 7   feature_6        131918 non-null  float64
 8   feature_7        132434 non-null  float64
 9   feature_8        132434 non-null  float64
 10  feature_9        132481 non-null  float64
 11  feature_10       132434 non-null  float64
 12  feature_11       132481 non-null  float64
 13  feature_12       132481 non-null  float64
 14  feature_13       132232 non-null  float64
 15  feature_14       132279 non-null  float64
 16  feature_15       132481 non-null  floa

In [115]:
train.describe()

,timestamp_(min),feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24
count,132481.000000,132481.000000,132481.000000,132481.000000,132481.000000,132279.000000,132232.000000,131918.000000,132434.000000,132434.000000,...,132481.000000,132481.000000,132481.000000,132279.000000,132481.000000,132481.000000,130345.000000,132277.000000,132481.000000,132434.000000
mean,66240.000000,0.835810,0.840205,0.634216,0.575081,0.543310,0.511753,0.549268,0.537706,0.551133,...,0.395703,0.552609,0.626023,0.456716,0.659005,0.009992,0.012716,0.264333,0.106294,0.171198
std,38244.114844,0.067326,0.039472,0.102920,0.049675,0.089438,0.077771,0.064093,0.090120,0.078996,...,0.040340,0.078733,0.050332,0.100087,0.033860,0.010383,0.015244,0.095716,0.125078,0.070909
min,0.000000,0.486490,0.525682,0.421484,0.380874,0.065106,0.110161,0.215909,0.077795,0.038693,...,0.126150,0.096146,0.459456,0.148545,0.502730,0.000000,0.000000,0.118326,0.000000,0.009217
25%,33120.000000,0.789497,0.820488,0.580170,0.543702,0.471135,0.451409,0.505682,0.470921,0.490750,...,0.366614,0.491537,0.585286,0.390680,0.638411,0.001686,0.000000,0.197355,0.017429,0.115207
50%,66240.000000,0.837290,0.843873,0.603304,0.572077,0.548428,0.507439,0.539773,0.529834,0.552253,...,0.393919,0.554260,0.613736,0.431563,0.652471,0.006745,0.007117,0.222880,0.047930,0.165899
75%,99360.000000,0.880603,0.866423,0.625805,0.611127,0.604875,0.559671,0.579545,0.588369,0.603570,...,0.424146,0.605177,0.659356,0.478530,0.670762,0.015177,0.024911,0.327563,0.165577,0.221198
max,132480.000000,1.000000,0.909956,1.000000,0.676682,0.929442,0.877493,1.000000,0.968278,0.935128,...,1.000000,0.934433,0.919789,1.000000,1.000000,1.000000,0.270463,1.000000,1.000000,1.000000


In [118]:
x_train.shape,y_train.shape, x_test.shape, y_test.shape

((132481, 25), (132481, 1), (87841, 25), (87841, 1))

In [119]:
y_test.apply(lambda x: x.unique())

,label
0,0
1,1


In [124]:
x_train -= x_train.min()
x_train /= x_train.max()

In [125]:
x_test -= x_test.min()
x_test /= x_test.max()

In [52]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers

# Define the MADE model
def made_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    # Define MADE architecture
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(input_dim)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Define the GAN generator
def generator_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    # Define GAN generator architecture
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(input_dim)(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Define the GAN discriminator
def discriminator_model(input_dim):
    inputs = tf.keras.Input(shape=(input_dim,))
    # Define GAN discriminator architecture
    x = layers.Dense(128, activation='relu')(inputs)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# Contrastive Loss
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def call(self, y_true, y_pred):
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(self.margin - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

# Train MADE model
def train_made(made, data, epochs=50, batch_size=32):
    made.compile(optimizer='adam', loss='mse')
    made.fit(data, data, epochs=epochs, batch_size=batch_size, verbose=0)

# Train GAN models
def train_gan(generator, discriminator, data, epochs=50, batch_size=32):
    gan_input = tf.keras.Input(shape=(25,))
    generated_data = generator(gan_input)
    gan_output = discriminator(generated_data)
    gan = tf.keras.Model(gan_input, gan_output)

    # Compile the GAN with contrastive loss
    contrastive_loss = ContrastiveLoss()
    gan.compile(optimizer='adam', loss=contrastive_loss)

    for epoch in range(epochs):
        generator_loss = []
        for _ in range(len(data) // batch_size):
            noise = np.random.randn(batch_size, 25)
            generated_data = generator.predict(noise)
            real_data = data[np.random.randint(0, len(data), batch_size)]
            X = np.concatenate([real_data, generated_data])
            y = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
            loss = gan.train_on_batch(X, y)
            generator_loss.append(loss)
        print("Epoch:", epoch+1, "Generator Loss:", np.mean(generator_loss))

# Anomaly detection using likelihood scores
def anomaly_detection(made, generator, data):
    # Calculate likelihood scores for real data
    real_likelihood = -tf.reduce_mean(made(data))

    # Generate synthetic data using GAN generator
    noise = np.random.randn(len(data), 25)
    generated_data = generator.predict(noise)

    # Calculate likelihood scores for generated data
    generated_likelihood = -tf.reduce_mean(made(generated_data))

    # Convert generated_likelihood to a 1-dimensional array
    generated_likelihood = np.atleast_1d(generated_likelihood)

    # Identify anomalies based on likelihood scores
    anomaly_indices = np.where(generated_likelihood < np.percentile(generated_likelihood, 5))[0]

    return anomaly_indices

# Main function
def main():
    # Load training data from CSV
    data = pd.read_csv("/content/RANSynCoders/data/train.csv")
    data = data.drop("timestamp_(min)", axis=1)
    data = data[:1000].values

    # Instantiate MADE model
    made = made_model(input_dim=25)

    # Train MADE model
    train_made(made, data, epochs=15)

    # Instantiate GAN generator and discriminator models
    generator = generator_model(input_dim=25)
    discriminator = discriminator_model(input_dim=25)

    # Train GAN models
    train_gan(generator, discriminator, data, epochs=20)


    test = pd.read_csv("/content/RANSynCoders/data/test.csv")
    test = test.drop("timestamp_(min)", axis=1)
    test = test.values
    # Anomaly detection
    anomaly_indices = anomaly_detection(made, generator, test)

    print("Anomalies detected at indices:", anomaly_indices)

if __name__ == "__main__":
    main()


1/1 [==============================] - 0s 17ms/step
Epoch: 1 Generator Loss: 0.053029128906843775
1/1 [==============================] - 0s 18ms/step
Epoch: 2 Generator Loss: 2.450656532543104e-10
1/1 [==============================] - 0s 25ms/step
Epoch: 3 Generator Loss: 1.9994115010588746e-11
1/1 [==============================] - 0s 17ms/step
Epoch: 4 Generator Loss: 1.844870793710834e-11
1/1 [==============================] - 0s 17ms/step
Epoch: 5 Generator Loss: 1.803200529641321e-11
1/1 [==============================] - 0s 18ms/step
Epoch: 6 Generator Loss: 1.958382311044762e-11
1/1 [==============================] - 0s 28ms/step
Epoch: 7 Generator Loss: 1.7586356010570028e-11
1/1 [==============================] - 0s 17ms/step
Epoch: 8 Generator Loss: 1.6257416448008502e-11
1/1 [==============================] - 0s 17ms/step
Epoch: 9 Generator Loss: 1.7844456367293253e-11
1/1 [==============================] - 0s 16ms/step
Epoch: 10 Generator Loss: 1.8894907508015456e-11
1/1 [